In [1]:
#images taken from https://unsplash.com/wallpapers
from random import randrange
images = ["1.jpg"]
totalParity=0
totalHamming=0
dataSize= {"parity":7,"hamming":4}
data = {"packet" : [] , "ptype" :[], "label" : []}

def createParity(bitstr,parity="even"):
    
    binaryCount = bitstr.count("1")
    if parity=="even":
        if binaryCount %2 ==0 : 
            returnValue = bitstr+"0"
        else :
            returnValue = bitstr+"1"
    else:
        # odd parity
        if binaryCount %2 ==1:
            returnValue = bitstr+"0"
        else : 
            returnValue = bitstr+"1"
    return returnValue

def createHamming(bitstr):
    #iam going to use hamming 15 11 means 4 bits parity 1 2 4 8 bits parity bits total bits are 
    '''
        p1 = 1 3 5 7 9 11 13 15
        p2 = 2 3 6 7 10 11 14 15
        p3 = 4 5 6 7 12 13 14 15
        p4 = 8 9 10 11 12 13 14 15
    '''
    packet = list(bitstr)
    packet.insert(0,"p")
    packet.insert(1,"p")
    packet.insert(3,"p")
    #packet.insert(7,"p")
    p1=""
    p2=""
    p3=""
    #p4=""
    for i in range(1,8):
    #for i in range(1,16):
        if bin(i)[2:].zfill(3)[-1]=="1" and i!=1:
            p1+=packet[i-1]
        if bin(i)[2:].zfill(3)[-2]=="1" and i!=2:
            p2+=packet[i-1]
        if bin(i)[2:].zfill(3)[-3]=="1" and i!=4:
            p3+=packet[i-1]
        #if bin(i)[2:].zfill(4)[-4]=="1" and i!=8:
        #    p4+=packet[i-1]
    packet[0] = "0" if p1.count("1")%2 ==0 else "1"
    packet[1] = "0" if p2.count("1")%2 ==0 else "1"
    packet[3] = "0" if p3.count("1")%2 ==0 else "1"
    #packet[7] = "0" if p4.count("1")%2 ==0 else "1"
    return ''.join(packet)




for image in images : 
    dosyaStart=0
    f=open(image,"rb")
    file_bytes = f.read()
    dosyaStop = len(file_bytes)*8 # converting bytes to bits
    bits = ''.join(format(byte, '08b') for byte in file_bytes)
    while dosyaStart < dosyaStop :
        label=0
        #myRandom = randrange(1,3)
        myRandom=2
        if myRandom ==1:
            totalParity+=1
            packet = bits[dosyaStart:dosyaStart+dataSize["parity"]]
            if len(packet)<dataSize["parity"]:
                break;
            dosyaStart+=dataSize["parity"]
            evenOrOdd = randrange(1,3)
            evenOrOdd=2
            if evenOrOdd == 1:
                ptype="odd"
                #packet = createParity(packet,"odd") sonuc lineer olmadigi icin sorun cikariyor
                packet = createParity(packet,"even")
            else : 
                ptype="even"
                packet = createParity(packet)
            if totalParity%2==0:
                label=1
                corruptbit = randrange(0,7)
                bit = packet[corruptbit]
                if bit == "1":
                    bit="0"
                else:
                    bit="1"
                packet=packet[:corruptbit] + bit + packet[corruptbit+1:]

        else : 
            totalHamming+=1
            #hamming code
            ptype="hamming"
            packet = bits[dosyaStart:dosyaStart+dataSize["hamming"]]
            dosyaStart+=dataSize["hamming"]
            if len(packet)<dataSize["hamming"]:
                break;
            packet=createHamming(packet)
            #datapackets = [2,4,5,6,8,9,10,11,12,13,14]
            datapackets = [2,4,5,6]
            if totalHamming%2==1:
                label=1
                corruptbit=datapackets[randrange(0,len(datapackets))]
                bit=packet[corruptbit]
                if bit=="1":
                    bit="0"
                else:
                    bit="1"
                packet=packet[:corruptbit] + bit + packet[corruptbit+1:]

        data["packet"].append(packet)
        data["ptype"].append(ptype)
        data["label"].append(label)
    f.close()





In [2]:
import pandas as pd
df=pd.DataFrame(data)
df = df.sample(frac=1, random_state=34).reset_index(drop=True)
train_size = int(0.1 * len(df))
traindf = df[:train_size]
testdf  = df[train_size:]
X_train = traindf["packet"].tolist()  # .tolist() ile bit string listesi
Y_train = traindf["label"].tolist()
X_test  = testdf["packet"].tolist()
Y_test  = testdf["label"].tolist()

print("Label distribution:", pd.Series(Y_train).value_counts())
traindf.head(15)

Label distribution: 0    367459
1    366710
Name: count, dtype: int64


,packet,ptype,label
0,1011010,hamming,0
1,0110111,hamming,1
2,1001100,hamming,0
3,1110001,hamming,1
4,0110001,hamming,1
5,0001110,hamming,1
6,0100101,hamming,0
7,0000000,hamming,0
8,0101010,hamming,0
9,0100101,hamming,0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ---------------- Positional Encoding ----------------
class PosEncoding(nn.Module):
    def __init__(self, d_model, max_len=15):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)


# ---------------- Multi-Head Attention ----------------
class Attention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.q = nn.Linear(d_model, d_model)
        self.k = nn.Linear(d_model, d_model)
        self.v = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        b, seq_len, d_model = x.size()
        Q = self.q(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1,2)
        scores = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(self.head_dim)
        if mask is not None:
            mask_exp = mask.unsqueeze(1).unsqueeze(2)
            scores = scores.masked_fill(mask_exp == 0, float('-inf'))
        attn = F.softmax(scores, dim=-1)
        out = torch.matmul(attn, V)
        out = out.transpose(1,2).contiguous().view(b, seq_len, d_model)
        return self.out(out)


# ---------------- Feed Forward ----------------
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))


# ---------------- Encoder Block ----------------
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attn = Attention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = FeedForward(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, mask=None):
        x = self.norm1(x + self.dropout(self.attn(x, mask)))
        x = self.norm2(x + self.dropout(self.ff(x)))
        return x


# ---------------- Bit Transformer ----------------
class BitTransformerClassifier(nn.Module):
    def __init__(self, d_model=128, num_heads=8, d_ff=512, num_layers=8,
                 num_classes=2, max_len=15, pad_idx=9):
        super().__init__()
        self.pad_idx = pad_idx
        vocab_size = 3
        self.embed = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_enc = PosEncoding(d_model, max_len)
        self.layers = nn.ModuleList([EncoderBlock(d_model, num_heads, d_ff) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        mask = (x != self.pad_idx)
        x = self.embed(x)
        x = self.pos_enc(x)
        for layer in self.layers:
            x = layer(x, mask)
        x = self.norm(x)
        mask_float = mask.unsqueeze(-1).float()
        x = (x * mask_float).sum(dim=1) / mask_float.sum(dim=1).clamp(min=1e-9)
        return self.classifier(x)


# ---------------- Focal Loss ----------------
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        if alpha is not None:
            self.alpha = torch.tensor(alpha, dtype=torch.float)
        else:
            self.alpha = None

    def forward(self, inputs, targets):
        device = inputs.device
        probs = F.softmax(inputs, dim=1).clamp(min=1e-9)
        pt = probs.gather(1, targets.unsqueeze(1)).squeeze(1)
        log_pt = torch.log(pt)
        if self.alpha is not None:
            alpha_t = self.alpha.to(device)[targets]  # <-- self.alpha GPU'ya taşındı
        else:
            alpha_t = torch.ones_like(pt, device=device)
        loss = -alpha_t * ((1 - pt) ** self.gamma) * log_pt
        return loss.mean()


# ---------------- Dataset Preparation ----------------
def prepare_tensor(bit_list, pad_idx=9, max_len=15):
    x = []
    for s in bit_list:
        bits = [int(b) for b in str(s).strip()]
        if len(bits) < max_len:
            bits += [pad_idx] * (max_len - len(bits))
        else:
            bits = bits[-max_len:]
        x.append(bits)
    return torch.tensor(x, dtype=torch.long)


# ---------------- Training Setup ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Model parametreleri
d_model = 64
num_heads = 4
d_ff = 128
num_layers = 4
epochs = 50
pad_idx = 2
max_len_input = 15
batch_size = 256

model = BitTransformerClassifier(d_model, num_heads, d_ff, num_layers,
                                 num_classes=2, max_len=max_len_input, pad_idx=pad_idx)
model.to(device)

# Class distribution
class_counts = Counter(Y_train)
total = sum(class_counts.values())
alpha_for_1 = 1 - (class_counts[1] / total)
print(f"Class distribution: {class_counts}, Focal alpha for class 1: {alpha_for_1:.4f}")

criterion = FocalLoss(alpha=[1-alpha_for_1, alpha_for_1], gamma=2.0)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)
steps_per_epoch = max(1, len(X_train)//batch_size)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=5e-4, epochs=epochs, steps_per_epoch=steps_per_epoch
)

X_train_tensor = prepare_tensor(X_train, pad_idx=pad_idx, max_len=max_len_input)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)

best_loss = float('inf')

# ---------------- Training Loop ----------------
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    for i in range(0, len(X_train_tensor), batch_size):
        x_batch = X_train_tensor[i:i+batch_size].to(device)
        y_batch = Y_train_tensor[i:i+batch_size].to(device)

        optimizer.zero_grad()
        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        running_loss += loss.item() * x_batch.size(0)
        _, predicted = torch.max(logits, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

    epoch_loss = running_loss / len(X_train_tensor)
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, zero_division=0)
    rec = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.6f} | Acc: {acc:.3f} | "
          f"Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f} | LR: {optimizer.param_groups[0]['lr']:.6f}")

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), "best_hamming74_model.pt")
        print("Best model saved!")



Using device: cuda
Class distribution: Counter({0: 367459, 1: 366710}), Focal alpha for class 1: 0.5005
Epoch 1/50 | Loss: 0.067168 | Acc: 0.685 | Prec: 0.749 | Rec: 0.557 | F1: 0.639 | LR: 0.000025
✅ Best model saved!
Epoch 2/50 | Loss: 0.056765 | Acc: 0.742 | Prec: 0.910 | Rec: 0.538 | F1: 0.676 | LR: 0.000041
✅ Best model saved!
Epoch 3/50 | Loss: 0.027959 | Acc: 0.911 | Prec: 0.984 | Rec: 0.836 | F1: 0.904 | LR: 0.000066
✅ Best model saved!
Epoch 4/50 | Loss: 0.002814 | Acc: 0.992 | Prec: 0.999 | Rec: 0.986 | F1: 0.992 | LR: 0.000099
✅ Best model saved!
Epoch 5/50 | Loss: 0.000144 | Acc: 1.000 | Prec: 1.000 | Rec: 1.000 | F1: 1.000 | LR: 0.000140
✅ Best model saved!


KeyboardInterrupt: 